In [84]:
import sys
sys.path.insert(0, '/Users/gulugulu/repos/PuningAnalysis/src')
from simulation.waiting_time_simulator import generate_ancestor, average_substitution, get_descrip_stat
from maths.evolutionary_rate import calculate_stationary_distribution, calculate_stationary_distribution, calculate_non_statioanry_mu
from cogent3.maths.matrix_exponential_integration import expected_number_subs
import numpy as np
import plotly.graph_objects as go
import pandas as pd
import plotly.express as px


In [22]:
#parameter set up
import json
with open('../../../data/dataset1_16s-10_initial_alignments/matrix_motif.json', 'r') as file:
    matrix_motif = json.load(file)


In [37]:
Q = {'0': np.array(matrix_motif['0']['matrix']['200580'])}
pi_0 = matrix_motif['0']['motif_prob']
pi_stationary = calculate_stationary_distribution(Q['0'])
n = 500
ances_seq = generate_ancestor(n, pi_0)
markov_order = 0

In [24]:
time_points = [0.5, 1, 5]

In [35]:
repeat_times = [100, 200, 500]

In [25]:
#theoretical value of ENS
theoretical_ns_list = []
for t in time_points:
    theoretical_ns = expected_number_subs(pi_0, Q['0'], t)
    theoretical_ns_list.append(theoretical_ns)


In [8]:
# simulated_ns_dict = {}
# for t in time_points:
#     for r in repeat_times:
#         print((t,r))
#         simulated_ns_dict[(t,r)] = average_substitution(Q, t, ances_seq, r, markov_order)

In [9]:
import json

In [10]:
with open('../../../results/evolution_time_test.json', 'r') as infile:
    convert_simulated_ns_dict = json.load(infile)

In [53]:
descrip_stat_time = get_descrip_stat(convert_simulated_ns_dict)

In [16]:
# import pandas as pd

# data = {
#     'Number of Substitution': convert_simulated_ns_dict['(0.5, 200)'][0] + convert_simulated_ns_dict['(1, 200)'][0] + convert_simulated_ns_dict['(5, 200)'][0],
#     'Time': ['0.5']*200 + ['1.0']*200 + ['5.0']*200
# }
# df = pd.DataFrame(data)

# # Create a violin plot with adjusted point positions
# fig = px.violin(df, y='Number of Substitution', x='Time', color='Time',
#                 box=True,  # Displays box plot inside the violin
#                 # points="all",  # Shows all points
#                 title="Violin Plot of Substitutions by Time",
#                 )

# fig.update_traces(jitter=0.05)  # Adjust jitter for better alignment
# fig.show()

In [17]:
# fig = px.histogram(x=ns_list, nbins=10)

# fig.update_layout(
#     title_text='Number Distribution',
#     xaxis_title_text='Number of Substitutions',
#     yaxis_title_text='Count',
#     bargap=0.1,
# )

# # Add a vertical line for the average
# fig.add_vline(x=a, line_width=3, line_dash="dash", line_color="red")
# fig.add_vline(x=average, line_width=3, line_dash="dash", line_color="lightgreen")
# fig.update_xaxes(range=[0.3, 0.5])
# fig.update_yaxes(range=[0, 40])

# fig.show()

In [106]:
simulated_ns_dict_repeat = {}
repeat_range2 = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 200, 300, 400]
for r in repeat_range2:
    print(r)
    simulated_ns_dict_repeat[r] = average_substitution(Q, 1, ances_seq, r, markov_order)

10
20
30
40
50
60
70
80
90
100
200
300
400


In [107]:
descrip_stat_repeat = get_descrip_stat(simulated_ns_dict_repeat)

In [108]:
descrip_stat_repeat[500] = descrip_stat_time['(1, 500)']

In [109]:
with open('../../../results/repetition_test.json', 'w') as outfile:
    json.dump(descrip_stat_repeat, outfile, indent=4)

In [110]:
# Extracting the repetitions (keys) and CVs (values) for plotting
repetitions = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 200, 300, 400, 500]
cvs = [info['coefficient of variation'] for info in descrip_stat_repeat.values()]

In [111]:
# Create the plot
fig = px.scatter(x=repetitions, y=cvs, labels={'x': 'Number of Repetitions', 'y': 'Coefficient of Variation'},
                 title='Coefficient of Variation vs. Number of Repetitions')

# Adding line for better trend visualization
fig.add_scatter(x=repetitions, y=cvs, mode='lines', showlegend=False)

# Show the plot
fig.show()

In [63]:
# Prepare data for plotting
reps = [100, 200, 500]
times = [0.5, 1, 5]
plot_data = {'Time': [], 'Repetitions': [], 'Coefficient of Variation': []}

for key, val in descrip_stat_time.items():
    time, rep = eval(key)
    plot_data['Time'].append(time)
    plot_data['Repetitions'].append(rep)
    plot_data['Coefficient of Variation'].append(val['coefficient of variation'])

df = pd.DataFrame(plot_data)

### Step 2: Create the Plot
fig2 = go.Figure()

# Add traces for each repetition count
for repeat in sorted(df['Repetitions'].unique()):
    sub_df = df[df['Repetitions'] == repeat]
    fig2.add_trace(go.Scatter(
        x=sub_df['Time'], 
        y=sub_df['Coefficient of Variation'],
        mode='lines+markers',
        name=f'Repetitions = {repeat}'
    ))

# Update layout
fig2.update_layout(
    title="Coefficient of Variation vs. Time for Different Repetitions",
    xaxis_title="Time",
    yaxis_title="Coefficient of Variation",
    legend_title="Number of Repetitions"
)

# Show the plot
fig2.show()

In [64]:
import plotly.graph_objects as go
import pandas as pd

# Prepare data for plotting
reps = [100, 200, 500]
times = [0.5, 1, 5]
plot_data = {'Time': [], 'Repetitions': [], 'Coefficient of Variation': []}

for key, val in descrip_stat_time.items():
    time, rep = eval(key)
    plot_data['Time'].append(time)
    plot_data['Repetitions'].append(rep)
    plot_data['Coefficient of Variation'].append(val['coefficient of variation'])

df = pd.DataFrame(plot_data)

### Step 2: Create the Plot
fig = go.Figure()

# Add traces for each time point
for time in times:
    sub_df = df[df['Time'] == time]
    fig.add_trace(go.Scatter(
        x=sub_df['Repetitions'], 
        y=sub_df['Coefficient of Variation'],
        mode='lines+markers',
        name=f'Time = {time}'
    ))

# Update layout
fig.update_layout(
    title="Coefficient of Variation vs. Number of Repetitions",
    xaxis_title="Number of Repetitions",
    yaxis_title="Coefficient of Variation",
    legend_title="Time Point"
)

# Show the plot
fig.show()

In [81]:
import plotly.express as px
import pandas as pd

ns_list_05 = convert_simulated_ns_dict['(0.5, 500)'][0]

fig3 = px.histogram(x=ns_list_05, nbins=10)

fig3.update_layout(
    title_text='Number of Substitution Distribution t=0.5',
    xaxis_title_text='Number of Substitutions',
    yaxis_title_text='Count',
    bargap=0.1,
)

# # Add a vertical line for the average
fig3.add_vline(x=convert_simulated_ns_dict['(0.5, 500)'][1], line_width=3, line_dash="dash", line_color="red")
fig3.add_vline(x=theoretical_ns_list[0], line_width=3, line_dash="dash", line_color="lightgreen")


fig3.show()

In [82]:
import plotly.express as px
import pandas as pd

ns_list_1 = convert_simulated_ns_dict['(1, 500)'][0]

fig4 = px.histogram(x=ns_list_1, nbins=10)

fig4.update_layout(
    title_text='Number of Substitution Distribution t=1',
    xaxis_title_text='Number of Substitutions',
    yaxis_title_text='Count',
    bargap=0.1,
)

# # Add a vertical line for the average
fig4.add_vline(x=convert_simulated_ns_dict['(1, 500)'][1], line_width=3, line_dash="dash", line_color="red")
fig4.add_vline(x=theoretical_ns_list[1], line_width=3, line_dash="dash", line_color="lightgreen")


fig4.show()

In [83]:
import plotly.express as px
import pandas as pd

ns_list_5 = convert_simulated_ns_dict['(5, 500)'][0]

fig5 = px.histogram(x=ns_list_5, nbins=10)

fig5.update_layout(
    title_text='Number of Substitution Distribution t=5',
    xaxis_title_text='Number of Substitutions',
    yaxis_title_text='Count',
    bargap=0.1,
)

# # Add a vertical line for the average
fig5.add_vline(x=convert_simulated_ns_dict['(5, 500)'][1], line_width=3, line_dash="dash", line_color="red")
fig5.add_vline(x=theoretical_ns_list[2], line_width=3, line_dash="dash", line_color="lightgreen")


fig5.show()

In [87]:
from scipy import stats
t_test_stat = {}
data = [convert_simulated_ns_dict['(0.5, 500)'][0], convert_simulated_ns_dict['(1, 500)'][0], convert_simulated_ns_dict['(5, 500)'][0]]
for i in range(3):
    time = time_points[i]
    sample_values = data[i]
    theoretical_mean = theoretical_ns_list[i]
    t_stat, p_value = stats.ttest_1samp(sample_values, theoretical_mean)
    t_test_stat[time] = [t_stat, p_value]


In [88]:
t_test_stat

{0.5: [-10.863661560516569, 8.032117805006006e-25],
 1: [-10.107061099527511, 5.6595811691594525e-22],
 5: [-2.4561703850969643, 0.014382330965653136]}